In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
HLA_associations_path = os.path.abspath("../")
sys.path.append(HLA_associations_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

import constants, index_tools, data_tools, medical_code_tools
from utilities import *

In [2]:
sns.set_palette("tab10")
plt.rcParams.update({'figure.max_open_warning': 0})

# Load BioBank Components

In [3]:
DEV_MODE = False
biobank_data, biobank_index, med_code_mapping = data_tools.load_all_biobank_components(DEV_MODE, signifier="cancer")

Importing BioBank Index and Data:
Missing 0 biobank index names
369
UK BioBank Data Loaded.
Size: 41700 rows x 369 columns
Elapsed time: 4.4412 seconds


Mapping ICD10 Codes:   0%|          | 0/315 [00:00<?, ? feature/s]

In [72]:
def get_unique_features_list(biobank_data: pd.DataFrame):
    """"""
    unique_features = {}
    for feature in biobank_data.columns:
        feature_base = "_".join(feature.split("_")[:-1]) if "." in feature else feature

        if feature_base not in unique_features:
            unique_features[feature_base] = feature

        else:
            saved_feature = unique_features[feature_base]
            if len(saved_feature) > len(feature):
                unique_features[feature_base] = feature

            elif len(saved_feature) == len(feature):
                if float(saved_feature.split("_")[-1]) > float(feature.split("_")[-1]):
                    unique_features[feature_base] = feature

    return list(unique_features.values())

unique_features = get_unique_features_list(biobank_data)

In [5]:
feature_bases = ["_".join(feature.split("_")[:-1]) if "." in feature else feature for feature in biobank_data.columns]

unique_features = {}
for feature in biobank_data.columns:
    feature_base = "_".join(feature.split("_")[:-1]) if "." in feature else feature

    if feature_base not in unique_features:
        unique_features[feature_base] = [feature]
    else:
        unique_features[feature_base].append(feature)

# unique_feature_bases, counts = np.unique(feature_bases, return_counts=True)
# counts

exception_feature_bases = ["genetic_PCs"]

multiple_features = {feature_base: features for feature_base, features in unique_features.items()
                     if len(features) > 1 and feature_base not in exception_feature_bases}
single_features = [features[0] for feature_base, features in unique_features.items() if len(features) == 1]
single_features += [feature for feature_base, features in unique_features.items() for feature in features
                    if feature_base in exception_feature_bases]

primary_features = [features[0] for features in multiple_features.values()]

secondary_features = {features[0]: features[1:] for features in multiple_features.values()}
secondary_features
len(secondary_features)

20

In [7]:
all_secondary_features = [feature for features in secondary_features.values() for feature in features]

In [10]:
secondary_feature_to_primary = {feature: features[0] for features in multiple_features.values() for feature in features}
secondary_feature_df = pd.DataFrame(dict([ (k, pd.Series(v)) for k, v in secondary_features.items() ]))

In [ ]:
TESTING_N = 1

secondary_feature_sets = secondary_feature_df.values.tolist()
secondary_feature_sets = [np.array([feature for feature in feature_set if not feature is np.nan])
                          for feature_set in secondary_feature_sets]
secondary_feature_set_indexs = [np.array([secondary_feature_to_primary[feature] for feature in feature_set])
                                        for feature_set in secondary_feature_sets]
primary_rows, secondary_rows = [], []
for i, row in tqdm(biobank_data.iterrows(), total=len(biobank_data), desc="Melting BioBank", unit=" People"):
    base_row = row[single_features + primary_features].copy(deep=True)
    primary_rows.append(base_row)
        
    for feature_set, feature_set_index in zip(secondary_feature_sets[:TESTING_N], secondary_feature_set_indexs[:TESTING_N]):
        valued_indices = ~ row[feature_set].isna().values
        if any(valued_indices):
            new_row = base_row.copy(deep=True)
            new_row[feature_set_index[valued_indices]] = row[feature_set[valued_indices]]
            secondary_rows.append(new_row)


Melting BioBank:   0%|          | 0/41700 [00:00<?, ? People/s]